In [2]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

2021.07.06

进行MPAS的后处理部分，计算每日降水的方差，并保存到同目录下的 output_nc 文件夹中，准备后续的NCL绘图

In [175]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_pre/mask_res/"
filename_cn051  = "mask_sel_CN05.1_Pre_1961_2018_daily_025x025.nc"
filename_cmorph = "mask_sel_CMORPH_98-17.nc"
filename_vr     = "mask_vr_pre_98-17.nc"
filename_rcm    = "mask_rcm_pre_98-17.nc"

ds_or = {}
ds_or['cn051']  = xr.open_dataset(dir_in + filename_cn051)
ds_or['cmorph'] = xr.open_dataset(dir_in + filename_cmorph)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['cn051'] = ds_or['cn051']['pre']
var['cmorph'] = ds_or['cmorph']['cmorph'][:, 0, :, :]
var['vr'] = ds_or['vr']['precip_MPAS']
var['rcm'] = ds_or['rcm']['precip_MPAS']
# change coords
var_list = ['cn051','cmorph', 'vr', 'rcm']
for i in var_list:
    rename_dict = dict(zip(var[i].coords.keys(), var['cn051'].coords.keys()))
    # show converting coords
    for rename_i in rename_dict:
        print(rename_i + " -----converting to----- " + rename_dict[rename_i])

    var[i] = var[i].rename(rename_dict)
    var[i]._coords = var['cn051']._coords
    var[i] = var[i].rename(i)


/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


time -----converting to----- time
lon -----converting to----- lon
lat -----converting to----- lat
time -----converting to----- time
lon -----converting to----- lon
lat -----converting to----- lat
Time -----converting to----- time
longitude -----converting to----- lon
latitude -----converting to----- lat
Time -----converting to----- time
longitude -----converting to----- lon
latitude -----converting to----- lat


In [178]:
# 将var拆分到am和jja
var_am  = {}
var_jja = {}
for ivar in var:
    var_am[ivar]  = var[ivar].sel(time = var[ivar].time.dt.month.isin([4,5]))
    var_jja[ivar] = var[ivar].sel(time = var[ivar].time.dt.month.isin([6,7,8]))
# 计算方差 dim[0] 即是时间维
variance_am  = {}
variance_jja = {}
for ind, ivar in enumerate(var_list):
    variance_am[ivar]  = var_am[ivar].var(dim=var_am[ivar].dims[0])
    variance_jja[ivar] = var_jja[ivar].var(dim=var_jja[ivar].dims[0])

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/core/nputils.py:152: RuntimeWarning: Degrees of freedom <= 0 

In [241]:
# check fillvalue
ds_am_variance['cn051'].fillna(-1e30)

<xarray.DataArray 'cn051' (lat: 160, lon: 280)>
array([[-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30],
       [-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30],
       [-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30],
       ...,
       [-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30],
       [-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30],
       [-1.e+30, -1.e+30, -1.e+30, ..., -1.e+30, -1.e+30, -1.e+30]],
      dtype=float32)
Coordinates:
  * lon      (lon) float32 70.12 70.38 70.62 70.88 ... 139.1 139.4 139.6 139.9
  * lat      (lat) float32 15.12 15.38 15.62 15.88 ... 54.12 54.38 54.62 54.88

In [258]:
# 合并字典中的计算结果到 ds_variance 输出到临时文件，准备后续NCL出图
ds_am_variance  = xr.merge(list(variance_am.values()) )
ds_jja_variance = xr.merge(list(variance_jja.values()) )
ds_am_variance.fillna(-1e30)
ds_jja_variance.fillna(-1e30)


dir_out = "/m2data2/yycheng/playground/MPAS_case_study/92-25km_vr_rcm_postprocess/visualization/check_obs/settled/pre/space/output_nc/"
# 添加每个变量的fillvalue
encoding = {}
for i in var_list:
    encoding[i] = {"_FillValue":-1e30}

ds_am_variance.to_netcdf(path = dir_out + "am_pre_variance.nc", encoding = encoding)
ds_jja_variance.to_netcdf(path = dir_out + "jja_pre_variance.nc", encoding = encoding)